Convert .mat to python-readable

In [ ]:
import pandas as pd
from scipy.io import loadmat
from datetime import datetime
from operator import add
from itertools import starmap
import glob

filenames = glob.glob(r"./*.mat")
dataTables = []

for filename in filenames:
    #filename = "SS2_Boom_Impact_Pressure_PSI_50Hz.mat"
    paramName = filename[6:-4]

    MatlabData = loadmat(filename)

    keys = list(MatlabData.keys())

    MatlabDataArrays = MatlabData[keys[-1]][0][0]

    matData = MatlabDataArrays[-1]
    matTime = MatlabDataArrays[-2]
    yearString = MatlabDataArrays[-11][0].split("-")[-1]
    endTimeString = yearString +":" + MatlabDataArrays[-15][0]+"000"
    startTimeString = yearString +":" + MatlabDataArrays[-16][0]+"000"

    dateFormat = '%Y:%j:%H:%M:%S.%f'

    startTime = datetime.strptime(startTimeString,dateFormat)
    endTime = datetime.strptime(endTimeString,dateFormat)

    duration = endTime - startTime
    nSamples = len(matData)
    dt = duration/nSamples
    dtArray = [dt*n for n in range(0,nSamples)]
    startTimeArray = [startTime for n in range(0,nSamples)]

    timeArray = list(starmap(add,zip(startTimeArray,dtArray)))

    dataLabels = ['time',paramName]

    matData = [data[0] for data in matData]

    dataTable = pd.DataFrame(columns=dataLabels)
    dataTable[dataLabels[0]] = timeArray
    dataTable[dataLabels[1]] = matData
    dataTable.set_index('time')
    dataTables.append(dataTable)

df = dataTables[0]

for dataTable in dataTables:
    df = df.merge(dataTable,on='time')

df.to_csv('./output_data.csv')